## DOWNLOAD DATA WITH EARTH-ENGINE
> Downloading Landsat imagery with earth-engine

In [3]:
# import ee
import os
import pandas as pd

In [ ]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [ ]:
NAME="wealth_index.csv"
S2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
PATH=os.path.join("..", NAME)
BANDS=['B4','B3','B2','B1','B5','B6','B7','B8']
CLOUD_COVER = 1
DIST=6200
OUTPUT_FOLDER='s2_export'

In [5]:
data=pd.read_csv(PATH)

**We loop over each area of interest (AOI) and download the least cloudy corresponding scene from Sentinel-2**

In [ ]:
for country in data["country"].unique():
    curr_country = data[ data["country"] == country ]
    for year in curr_country["year"].unique():
        curr_year = curr_country[ data["year"] == year ]
        for _, row in curr_year.iterrows():
            aoi = ee.Geometry.Point(row['lat'], row['lon']).buffer(DIST)
            start_date = str(year)+'-01-01'
            end_date   = str(year)+'-12-31'
            filtered_collection = S2.filterBounds(aoi).filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1))
            mosaic = filtered_collection.median().select(BANDS).clip(aoi)
            task = ee.batch.Export.image.toDrive(image=mosaic,
                                     description='S2_tile_'+year+'_'+country,
                                     scale=30,
                                     region=aoi,
                                     fileNamePrefix=aoi+year+'_'+country,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
            task.start()


In [ ]:
# sentinel = ee.ImageCollection('COPERNICUS/S2_SR') \
#     .filterDate('2020-04-01','2021-10-01') \
#     .filterBounds(geometry) \
#     .map(clipcol) \
#     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) 

# ndvi = sentinel.map(calculateNDVI)
# maxNDVI = ndvi.max()
# geemap.ee_export_image(maxNDVI, 
#                        filename= maxNDVI.tif,
#                        scale = 10, region = geometry, file_per_band=False,
#                        crs = 'EPSG:3035')